In [ ]:
from lightgbm import LGBMRegressor
from lightgbm import plot_importance as lgbm_plot_importance
import matplotlib.pyplot as plt
from matplotlib import gridspec
import numpy as np
import pandas as pd
import pytz
import seaborn as sns
from sklearn.metrics import mean_absolute_error, root_mean_squared_error
from sklearn.linear_model import LinearRegression

In [ ]:
# for plotting
sns.set_style("darkgrid")

df = pd.read_csv('SG.csv', sep=';', parse_dates = ['Time'])
df['Time'] = pd.to_datetime(df['Time'], utc=True)

df['summer_time'] = df['Time'].dt.tz_convert("Europe/Bratislava").dt.hour - df['Time'].dt.hour - 1
df['summer_time'] = df['summer_time'].apply(lambda x: 0 if x == -24  else (1 if x == -23 else x))  # -> only values 0/1

df['Time'] = df['Time'].dt.tz_localize(None)
df.set_index('Time', inplace=True)
df.loc[:, 'date'] = df.index.date

df['batt_power'] = df['Battery charging'] - df['Battery discharging']
df['grid_power'] = df['Grid consumption'] - df['Grid backflow']

df.loc[:, 'Consumption'] = df.loc[:, 'Consumption'].fillna((df.loc[:, 'Consumption'].shift(48) + df.loc[:, 'Consumption'].shift(-48)) / 2)
df.loc[:, 'PV generation'] = df.loc[:, 'PV generation'].fillna((df.loc[:, 'PV generation'].shift(48) + df.loc[:, 'PV generation'].shift(-48)) / 2)
df.loc[:, 'batt_power'] = df.loc[:, 'batt_power'].fillna((df.loc[:, 'batt_power'].shift(48) + df.loc[:, 'batt_power'].shift(-48)) / 2)
df['grid_power'] = df['grid_power'].fillna((df['Consumption'] - df['PV generation'] + df['batt_power']))

# df = df.interpolate(method='polynomial', order=3, limit_direction='both', axis=0)

df['period_of_day'] = df.groupby(['date'])[df.columns.to_list()].cumcount()
df['batt_power_cumsum'] = df['batt_power'].cumsum()

# lepsie by bolo to len rozdelit na kladne a zaporne hodnoty...
df.loc[:, 'Grid backflow'] = df.loc[:, 'Grid backflow'].fillna(0)
df.loc[:, 'Grid consumption'] = df.loc[:, 'Grid consumption'].fillna(df.loc[:, 'grid_power'] - df.loc[:, 'Grid backflow']) 

# fillna day rows
df.loc[df['period_of_day'] >= 10, 'Battery charging'] = df.loc[df['period_of_day'] >= 10, 'Battery charging'].fillna(0)
df.loc[df['period_of_day'] >= 10, 'Battery discharging'] = df.loc[df['period_of_day'] >= 10, 'Battery discharging'].fillna(df['batt_power'] - df['Battery charging'])
# fillna night rows
df.loc[df['period_of_day'] < 10, 'Battery discharging'] = df.loc[df['period_of_day'] < 10, 'Battery discharging'].fillna(0)
df.loc[df['period_of_day'] < 10, 'Battery charging'] = df.loc[df['period_of_day'] < 10, 'Battery charging'].fillna(df['batt_power'] - df['Battery discharging'])


# look on the slice with filled NaN values
df.loc[pd.to_datetime('2022-03-18 23:00:00'):pd.to_datetime('2022-03-19 02:00:00'), :]

In [ ]:
df['batt_power_cumsum'].plot()
# upwards trend - battery always draws more power to charge than it discharges due to efficiency/losses

In [ ]:
# remove the trend 
num_range = np.arange(len(df['batt_power_cumsum'])).reshape(-1, 1)
batt_power_cumsum = df['batt_power_cumsum']

m = LinearRegression()
m.fit(num_range, batt_power_cumsum)

print(m.coef_)
print(m.intercept_)

plt.plot(num_range, batt_power_cumsum,color='g')
plt.plot(num_range, m.predict(num_range),color='k')
plt.plot(num_range, m.coef_ * num_range,color='r')

plt.show()

In [ ]:
print((df['batt_power_cumsum'] - (m.coef_ * num_range.reshape(-1))).min())
print((df['batt_power_cumsum'] - (m.coef_ * num_range.reshape(-1))).max())

(df['batt_power_cumsum'] - (m.coef_ * num_range.reshape(-1))).plot()

# Feature engineering

In [ ]:
df.columns

In [ ]:
df['batt_p_cumsum_detrend'] = df['batt_power_cumsum'] - (m.coef_ * num_range.reshape(-1))   
df['batt_p_cumsum_detrend_shift1'] = df['batt_p_cumsum_detrend'].shift(1) # available capacity to charge/discharge

df['PV generation_shift1'] = df['PV generation'].shift(1)
df['Consumption_shift1'] = df['Consumption'].shift(1)
df['grid_power_shift1'] = df['grid_power'].shift(1)

df['Consumption_shift48'] = df['Consumption'].shift(48)
df['PV generation_shift48'] = df['PV generation'].shift(48)

df = df.dropna()

df

# LightGBM regressor

## Simple fit & predict on the whole dataset, no train-test split  
## Mnual feature selection

In [ ]:
list(df.columns)

In [ ]:
# manual feature selection -> 3 input set for each target variable
df_y_consumption = df[['Consumption']]
df_X_consumption1 = df[['period_of_day', 'PV generation_shift1', 'Consumption_shift1', ]]
df_X_consumption2 = df[['period_of_day', 'PV generation_shift1', 'Consumption_shift1', 'Consumption_shift48']]
df_X_consumption3 = df[['period_of_day', 'PV generation_shift1', 'Consumption_shift1', 'Consumption_shift48', 'summer_time']]

df_y_PV_gen = df[['PV generation']]
df_X_PV_gen1 = df[['period_of_day', 'PV generation_shift1']]
df_X_PV_gen2 = df[['period_of_day', 'PV generation_shift48']]
df_X_PV_gen3 = df[['period_of_day', 'PV generation_shift1', 'PV generation_shift48']]

df_y_batt_power = df[['batt_power']]
df_X_batt_power1 = df[['period_of_day', 'PV generation_shift1', 'Consumption_shift1']]
df_X_batt_power2 = df[['period_of_day', 'batt_p_cumsum_detrend_shift1', 'PV generation_shift1', 'Consumption_shift1']]
df_X_batt_power3 = df[['period_of_day', 'batt_p_cumsum_detrend_shift1', 'PV generation_shift1', 'Consumption_shift1', 'summer_time']]

df_y_grid_power = df[['grid_power']]
df_X_grid_power1 = df[['period_of_day', 'grid_power_shift1', 'PV generation_shift1']]
df_X_grid_power2 = df[['period_of_day', 'batt_p_cumsum_detrend_shift1', 'grid_power_shift1', 'PV generation_shift1']]
df_X_grid_power3 = df[['period_of_day', 'batt_p_cumsum_detrend_shift1', 'grid_power_shift1', 'PV generation_shift1', 'summer_time']]

list_X_y1 = [(df_X_consumption1, df_y_consumption), (df_X_PV_gen1, df_y_PV_gen), (df_X_batt_power1, df_y_batt_power), (df_X_grid_power1, df_y_grid_power)]
list_X_y2 = [(df_X_consumption2, df_y_consumption), (df_X_PV_gen2, df_y_PV_gen), (df_X_batt_power2, df_y_batt_power), (df_X_grid_power2, df_y_grid_power)]
list_X_y3 = [(df_X_consumption3, df_y_consumption), (df_X_PV_gen3, df_y_PV_gen), (df_X_batt_power3, df_y_batt_power), (df_X_grid_power3, df_y_grid_power)]

In [ ]:
def lgbm_fit_predict_all4(list_X_y: list, df_source: pd.DataFrame) -> (pd.DataFrame, list):
    """"""
    df_predictions = df_source[['period_of_day']].copy()
    list_quantity_names = []
    list_models = []
    
    for X, y in list_X_y:
        lgbm_reg = LGBMRegressor(objective='regression')
        lgbm_reg.fit(X, y)
        prediction = lgbm_reg.predict(X)  
        
        df_predictions[f'{y.columns[0]}_true'] = y.values
        df_predictions[f'{y.columns[0]}_predicted'] = prediction
        df_predictions[f'{y.columns[0]}_residuals'] = np.subtract(y.values.reshape(-1), prediction)
        
        list_quantity_names.append(y.columns[0])
        list_models.append(lgbm_reg)
        
    return df_predictions, list_quantity_names, list_models
        
        
def plot_predictions(df_predictions: pd.DataFrame, list_quantity_names: list) -> None:
    """"""
    plt.figure(figsize=(30, 20))
    for q in range(len(list_quantity_names)):
        q_true = f'{list_quantity_names[q]}_true'
        q_predict = f'{list_quantity_names[q]}_predicted'        
        
        ax = plt.subplot(4, 1, q+1)
        ax.plot(df_predictions.index, df_predictions.loc[:, q_true], label='true')
        ax.plot(df_predictions.index, df_predictions.loc[:, q_predict], label='predicted')
        ax.legend()
        ax.title.set_text(
            f'{list_quantity_names[q]}, '
            f'mae = {np.round(mean_absolute_error(df_predictions.loc[:, q_true], df_predictions.loc[:, q_predict]), 2)}, '
            f'rmse = {np.round(root_mean_squared_error(df_predictions.loc[:, q_true], df_predictions.loc[:, q_predict]), 2)}'
            )
    plt.show()


def plot_residuals_pt_feat_imp(df_predictions: pd.DataFrame, list_quantity_names: list, list_models: list) -> None:
    """"""
    df_predict = df_predictions.copy()
    df_predict['date'] = df_predict.index.date
    
    fig = plt.figure(figsize=(30, 20))
    gs = gridspec.GridSpec(4, 3, width_ratios=[3, 1, 1]) 
    for i in range(0, 3*len(list_quantity_names), 3):
        q=int(i/3)
        q_true = f'{list_quantity_names[q]}_true'
        q_predict = f'{list_quantity_names[q]}_predicted'  
        q_res = f'{list_quantity_names[q]}_residuals'
        
        # another way of creating stacked array for boxplot
        df_predict_pivot = df_predict.loc[:, ['period_of_day', q_res, 'date']].pivot_table(index='date', columns='period_of_day')
        stack_array = np.array(df_predict_pivot.values)
        # boxplot does not accept columns beginning or ending with zeros -> exclude incomplete days if present at the beginning or end
        if np.any(np.isnan(stack_array[0, :])):
            stack_array = stack_array[1:, :]
        if np.any(np.isnan(stack_array[-1, :])):
            stack_array = stack_array[:-1, :]
        
        ax1 = fig.add_subplot(gs[i]) # fig.add_subplot(4, 3, i+1)
        ax1.boxplot(stack_array)
        ax1.set_xlabel("day period")
        ax1.set_ylabel("residuals")
        ax1.set_title(f"Residuals per period of day for: {list_quantity_names[q]}")
        
        ax2 = fig.add_subplot(gs[i+1]) # fig.add_subplot(4, 3, i+2)
        ax2.scatter(df_predictions.loc[:, q_true], df_predictions.loc[:, q_predict], label="data")
        line45 = np.linspace(min(df_predictions.loc[:, q_true].values), max(df_predictions.loc[:, q_true].values))
        ax2.plot(line45, line45, color='red', linestyle='--', lw=3, label='ideal line')
        ax2.set_xlabel("True")
        ax2.set_ylabel("Predicted")
        ax2.legend()
        
        ax3 = fig.add_subplot(gs[i+2]) # fig.add_subplot(4, 3, i+3, gs[q])
        lgbm_plot_importance(list_models[q], ax=ax3, title=f"{list_quantity_names[q]}")

    plt.subplots_adjust(wspace=0.2)
    plt.show() 

#### Feature set 1

In [ ]:
df_predict1, list_q_n1, list_regr_models1 = lgbm_fit_predict_all4(list_X_y1, df)
plot_predictions(df_predict1, list_q_n1)

In [ ]:
plot_residuals_pt_feat_imp(df_predict1, list_q_n1, list_regr_models1)

#### Feature set 2

In [ ]:
df_predict2, list_q_n2, list_regr_models2 = lgbm_fit_predict_all4(list_X_y2, df)
plot_predictions(df_predict2, list_q_n2)

In [ ]:
plot_residuals_pt_feat_imp(df_predict2, list_q_n2, list_regr_models2)

#### Feature set 3

In [ ]:
df_predict3, list_q_n3, list_regr_models3 = lgbm_fit_predict_all4(list_X_y3, df)
plot_predictions(df_predict3, list_q_n3)

In [ ]:
plot_residuals_pt_feat_imp(df_predict3, list_q_n3, list_regr_models3)

- bool variable summer_time does not add much information to the model